In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer

In [4]:
h2o.init()
train =  h2o.import_file('sub_train.csv')
test  =  h2o.import_file('sub_test.csv')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqo22pxu3
  JVM stdout: /tmp/tmpqo22pxu3/h2o_hieunm_started_from_python.out
  JVM stderr: /tmp/tmpqo22pxu3/h2o_hieunm_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Asia/Ho_Chi_Minh
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,"28 days, 4 hours and 20 minutes"
H2O_cluster_name:,H2O_from_python_hieunm_6ujggl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.97 Gb
H2O_cluster_total_cores:,48
H2O_cluster_allowed_cores:,48
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
target=["contest-tmp2m-14d__tmp2m"]

In [8]:
x = train.columns
x.remove(target[0])
x.remove("startdate")

In [ ]:
aml = H2OAutoML(max_runtime_secs= (3600 * 12), 
                seed=1,stopping_metric='RMSE',
                max_models=None, 
                include_algos = ["GLM", "DeepLearning", "DRF","XGBoost","GBM","StackedEnsemble"])

aml.train(x=x, y=target[0], training_frame=train)

# Get AutoML event log
log = aml.event_log

# Get training timing info
info = aml.training_info

AutoML progress: |██████████████████████████████████████████████████████████████

In [10]:
log

NameError: name 'log' is not defined

In [ ]:
info

In [ ]:
aml.leaderboard.head(10)

In [ ]:
preds = aml.predict(test)

In [ ]:
res_df = pd.DataFrame(columns = [target[0], 'index'])
res_df['index'] = h2o.as_list(test['index'])['index']
res_df[target[0]] = list(h2o.as_list(preds)['predict'])

In [ ]:
res_df.to_csv('./submission_automl_v1.csv', index=False)
self = res_df[target[0]]
best_df = pd.read_csv('./best_sub.csv')
y_best = best_df[target[0]].values
self = t[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, self)/(norm(y_best)*norm(self))
print("cos_sim with best submission:", cos_sim)